<a href="https://colab.research.google.com/github/garci843/mgmt467-analytics-portfolio/blob/main/Unit2_Lab1_PromptPlusExamples_Colab_Kaggle_GCS_BQ_DQ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MGMT 467 — Prompt-Driven Lab (with Commented Examples)
## Kaggle ➜ Google Cloud Storage ➜ BigQuery ➜ Data Quality (DQ)

**How to use this notebook**
- Each section gives you a **Build Prompt** to paste into Gemini/Vertex AI (or Gemini in Colab).
- Below each prompt, you’ll see a **commented example** of what a good LLM answer might look like.
- **Do not** just uncomment and run. Use the prompt to generate your own code, then compare to the example.
- After every step, run the **Verification Prompt**, and write the **Reflection** in Markdown.

> Goal today: Download the Netflix dataset (Kaggle) → Stage on GCS → Load into BigQuery → Run DQ profiling (missingness, duplicates, outliers, anomaly flags).


### Academic integrity & LLM usage
- Use the prompts here to generate your own code cells.
- Read concept notes and write the reflection answers in your own words.
- Keep credentials out of code. Upload `kaggle.json` when asked.


## Learning objectives
1) Explain **why** we stage data in GCS and load it to BigQuery.  
2) Build an **idempotent**, auditable pipeline.  
3) Diagnose **missingness**, **duplicates**, and **outliers** and justify cleaning choices.  
4) Connect DQ decisions to **business/ML impact**.


## 0) Environment setup — What & Why
Authenticate Colab to Google Cloud so we can use `gcloud`, GCS, and BigQuery. Set **PROJECT_ID** and **REGION** once for consistency (cost/latency).

### Build Prompt (paste to LLM)
You are my cloud TA. Generate a single **Colab code cell** that:
1) Authenticates to Google Cloud in Colab,  
2) Prompts for `PROJECT_ID` via `input()` and sets `REGION="us-central1"` (editable),  
3) Exports `GOOGLE_CLOUD_PROJECT`,  
4) Runs `gcloud config set project $GOOGLE_CLOUD_PROJECT`,  
5) Prints both values. Add 2–3 comments explaining what/why.
End with a comment: `# Done: Auth + Project/Region set`.


In [ ]:
from google.colab import auth
auth.authenticate_user()


In [ ]:
import os
PROJECT_ID = input("Enter your GCP Project ID: ").strip()
REGION = "us-central1"  # keep consistent; change if instructed
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
print("Project:", PROJECT_ID, "| Region:", REGION)

# Set active project for gcloud/BigQuery CLI
!gcloud config set project $GOOGLE_CLOUD_PROJECT
!gcloud config get-value project
# Done: Auth + Project/Region set

Enter your GCP Project ID: mgmt467-4889
Project: mgmt467-4889 | Region: us-central1
Updated property [core/project].
mgmt467-4889


### Verification Prompt
Generate a short cell that prints the active project using `gcloud config get-value project` and echoes the `REGION` you set.


In [ ]:
# Print the active project and echo the region
!gcloud config get-value project
print(f"Region: {REGION}")

mgmt467-4889
Region: us-central1


**Reflection:** Why do we set `PROJECT_ID` and `REGION` at the top? What can go wrong if we don’t?

We set `PROJECT_ID` and `REGION` at the top because it keeps everything consistent and running in the proper place. If we don't our resources might accidentally get spread across different projects or regions which can lead to poor performance, incomplete work, and accidental billing.


## 1) Kaggle API — What & Why
Use Kaggle CLI for reproducible downloads. Store `kaggle.json` at `~/.kaggle/kaggle.json` with `0600` permissions to protect secrets.

### Build Prompt
Generate a **single Colab code cell** that:
- Prompts me to upload `kaggle.json`,
- Saves to `~/.kaggle/kaggle.json` with `0600` permissions,
- Prints `kaggle --version`.
Add comments about security and reproducibility.


In [ ]:
# kaggle CLI for reproducible downloads
from google.colab import files
import os

# Prompt user to upload kaggle.json
print("Upload your kaggle.json (Kaggle > Account > Create New API Token)")
uploaded = files.upload()

# Create directory and save the file with secure permissions
os.makedirs('/root/.kaggle', exist_ok=True)
with open('/root/.kaggle/kaggle.json', 'wb') as f:
    f.write(uploaded[list(uploaded.keys())[0]])
os.chmod('/root/.kaggle/kaggle.json', 0o600)  # Set owner-only read/write permissions

# Verify Kaggle CLI is installed and configured
!kaggle --version

Upload your kaggle.json (Kaggle > Account > Create New API Token)


Saving kaggle (1).json to kaggle (1).json
Kaggle API 1.7.4.5


### Verification Prompt
Generate a one-liner that runs `kaggle --help | head -n 20` to show the CLI is ready.


In [ ]:
#Generate a one-liner that runs kaggle --help | head -n 20 to show the CLI is ready.
!kaggle --help | head -n 20

usage: kaggle [-h] [-v] [-W]
              {competitions,c,datasets,d,kernels,k,models,m,files,f,config}
              ...

options:
  -h, --help            show this help message and exit
  -v, --version         Print the Kaggle API version
  -W, --no-warn         Disable out-of-date API version warning

commands:
  {competitions,c,datasets,d,kernels,k,models,m,files,f,config}
                        Use one of:
                        competitions {list, files, download, submit, submissions, leaderboard}
                        datasets {list, files, download, create, version, init, metadata, status}
                        kernels {list, files, init, push, pull, output, status}
                        models {instances, get, list, init, create, delete, update}
                        models instances {versions, get, files, init, create, delete, update}
                        models instances versions {init, create, download, delete, files}
                        config {view, set,

**Reflection:** Why require strict `0600` permissions on API tokens? What risks are we avoiding?

We require strict `0600` permissions on API tokens to prevent unauthorized access to our sensitive credentials. This ensures that only we, the owners of the file, can read or modify it, reducing the risk of accidental exposure or malicious use by others.

## 2) Download & unzip dataset — What & Why
Keep raw files under `/content/data/raw` for predictable paths and auditing.
**Dataset:** `sayeeduddin/netflix-2025user-behavior-dataset-210k-records`

### Build Prompt
Generate a **Colab code cell** that:
- Creates `/content/data/raw`,
- Downloads the dataset to `/content/data` with Kaggle CLI,
- Unzips into `/content/data/raw` (overwrite OK),
- Lists all CSVs with sizes in a neat table.
Include comments describing each step.


In [ ]:
!mkdir -p /content/data/raw
!kaggle datasets download -d sayeeduddin/netflix-2025user-behavior-dataset-210k-records -p /content/data
!unzip -o /content/data/*.zip -d /content/data/raw
# List CSV inventory
!ls -lh /content/data/raw/*.csv

Dataset URL: https://www.kaggle.com/datasets/sayeeduddin/netflix-2025user-behavior-dataset-210k-records
License(s): CC0-1.0
  0% 0.00/4.02M [00:00<?, ?B/s]
100% 4.02M/4.02M [00:00<00:00, 790MB/s]
Archive:  /content/data/netflix-2025user-behavior-dataset-210k-records.zip
  inflating: /content/data/raw/README.md  
  inflating: /content/data/raw/movies.csv  
  inflating: /content/data/raw/recommendation_logs.csv  
  inflating: /content/data/raw/reviews.csv  
  inflating: /content/data/raw/search_logs.csv  
  inflating: /content/data/raw/users.csv  
  inflating: /content/data/raw/watch_history.csv  
-rw-r--r-- 1 root root 114K Aug  2 19:36 /content/data/raw/movies.csv
-rw-r--r-- 1 root root 4.5M Aug  2 19:36 /content/data/raw/recommendation_logs.csv
-rw-r--r-- 1 root root 1.8M Aug  2 19:36 /content/data/raw/reviews.csv
-rw-r--r-- 1 root root 2.2M Aug  2 19:36 /content/data/raw/search_logs.csv
-rw-r--r-- 1 root root 1.6M Aug  2 19:36 /content/data/raw/users.csv
-rw-r--r-- 1 root root 8.9M A

### Verification Prompt
Generate a snippet that asserts there are exactly **six** CSV files and prints their names.


In [ ]:
# assert that 6 csv files exist and print their names
import glob
csv_files = glob.glob('/content/data/raw/*.csv')
assert len(csv_files) == 6, f"Expected 6 CSV files, but found {len(csv_files)}"
print("CSV files:")
for file in csv_files:
    print(file)

CSV files:
/content/data/raw/movies.csv
/content/data/raw/watch_history.csv
/content/data/raw/search_logs.csv
/content/data/raw/users.csv
/content/data/raw/recommendation_logs.csv
/content/data/raw/reviews.csv


**Reflection:** Why is keeping a clean file inventory (names, sizes) useful downstream?

Keeping a clean file inventory makes your code a lot easier to read when referring to different files and brings consistency and adaptability. It's also useful to have consistent names and sizes since you only have to remember a few things at once.

## 3) Create GCS bucket & upload — What & Why
Stage in GCS → consistent, versionable source for BigQuery loads. Bucket names must be **globally unique**.

### Build Prompt
Generate a **Colab code cell** that:
- Creates a unique bucket in `${REGION}` (random suffix),
- Saves name to `BUCKET_NAME` env var,
- Uploads all CSVs to `gs://$BUCKET_NAME/netflix/`,
- Prints the bucket name and explains staging benefits.


In [ ]:
import uuid, os
bucket_name = f"mgmt467-netflix-{uuid.uuid4().hex[:8]}"
os.environ["BUCKET_NAME"] = bucket_name
!gcloud storage buckets create gs://$BUCKET_NAME --location=US
!gcloud storage cp /content/data/raw/* gs://$BUCKET_NAME/netflix/
print("Bucket:", bucket_name)
# Verify contents
!gcloud storage ls gs://$BUCKET_NAME/netflix/

Creating gs://mgmt467-netflix-fcfa1523/...
Copying file:///content/data/raw/movies.csv to gs://mgmt467-netflix-fcfa1523/netflix/movies.csv
Copying file:///content/data/raw/README.md to gs://mgmt467-netflix-fcfa1523/netflix/README.md
Copying file:///content/data/raw/recommendation_logs.csv to gs://mgmt467-netflix-fcfa1523/netflix/recommendation_logs.csv
Copying file:///content/data/raw/reviews.csv to gs://mgmt467-netflix-fcfa1523/netflix/reviews.csv
Copying file:///content/data/raw/search_logs.csv to gs://mgmt467-netflix-fcfa1523/netflix/search_logs.csv
Copying file:///content/data/raw/users.csv to gs://mgmt467-netflix-fcfa1523/netflix/users.csv
Copying file:///content/data/raw/watch_history.csv to gs://mgmt467-netflix-fcfa1523/netflix/watch_history.csv

Average throughput: 23.5MiB/s
Bucket: mgmt467-netflix-fcfa1523
gs://mgmt467-netflix-fcfa1523/netflix/README.md
gs://mgmt467-netflix-fcfa1523/netflix/movies.csv
gs://mgmt467-netflix-fcfa1523/netflix/recommendation_logs.csv
gs://mgmt467-n

In [ ]:
# List objects and their sizes in the netflix/ prefix
!gcloud storage ls -l gs://{os.environ['BUCKET_NAME']}/netflix/

      8002  2025-10-26T15:33:32Z  gs://mgmt467-netflix-fcfa1523/netflix/README.md
    115942  2025-10-26T15:33:32Z  gs://mgmt467-netflix-fcfa1523/netflix/movies.csv
   4695557  2025-10-26T15:33:32Z  gs://mgmt467-netflix-fcfa1523/netflix/recommendation_logs.csv
   1861942  2025-10-26T15:33:32Z  gs://mgmt467-netflix-fcfa1523/netflix/reviews.csv
   2250902  2025-10-26T15:33:32Z  gs://mgmt467-netflix-fcfa1523/netflix/search_logs.csv
   1606820  2025-10-26T15:33:32Z  gs://mgmt467-netflix-fcfa1523/netflix/users.csv
   9269425  2025-10-26T15:33:32Z  gs://mgmt467-netflix-fcfa1523/netflix/watch_history.csv
TOTAL: 7 objects, 19808590 bytes (18.89MiB)


### Verification Prompt
Generate a snippet that lists the `netflix/` prefix and shows object sizes.


In [ ]:
import uuid, os
bucket_name = f"mgmt467-netflix-{uuid.uuid4().hex[:8]}"
os.environ["BUCKET_NAME"] = bucket_name
!gcloud storage buckets create gs://$BUCKET_NAME --location=US
!gcloud storage cp /content/data/raw/* gs://$BUCKET_NAME/netflix/
print("Bucket:", bucket_name)
# Verify contents
!gcloud storage ls gs://$BUCKET_NAME/netflix/

Creating gs://mgmt467-netflix-237da3ea/...
Copying file:///content/data/raw/movies.csv to gs://mgmt467-netflix-237da3ea/netflix/movies.csv
Copying file:///content/data/raw/README.md to gs://mgmt467-netflix-237da3ea/netflix/README.md
Copying file:///content/data/raw/recommendation_logs.csv to gs://mgmt467-netflix-237da3ea/netflix/recommendation_logs.csv
Copying file:///content/data/raw/reviews.csv to gs://mgmt467-netflix-237da3ea/netflix/reviews.csv
Copying file:///content/data/raw/search_logs.csv to gs://mgmt467-netflix-237da3ea/netflix/search_logs.csv
Copying file:///content/data/raw/users.csv to gs://mgmt467-netflix-237da3ea/netflix/users.csv
Copying file:///content/data/raw/watch_history.csv to gs://mgmt467-netflix-237da3ea/netflix/watch_history.csv

Average throughput: 13.5MiB/s
Bucket: mgmt467-netflix-237da3ea
gs://mgmt467-netflix-237da3ea/netflix/README.md
gs://mgmt467-netflix-237da3ea/netflix/movies.csv
gs://mgmt467-netflix-237da3ea/netflix/recommendation_logs.csv
gs://mgmt467-n

**Reflection:** Name two benefits of staging in GCS vs loading directly from local Colab.

Staging our data in Google Cloud Storage before loading it into BigQuery gives us a couple of cool advantages compared to just pulling it straight from our Colab notebook. First off, it makes things super consistent and easy to track. We know exactly which version of the data we're working with in GCS, which is great for making sure our BigQuery loads are always the same if we need to re-run things. It's like having a reliable checkpoint for our data. Second, it's way more practical for bigger datasets. GCS is built for handling tons of data, and it connects seamlessly with BigQuery. Trying to load huge files directly from Colab can be slow and clunky, but using GCS makes the whole process smoother and lets us work with much larger datasets easily within the Google Cloud ecosystem.

## 4) BigQuery dataset & loads — What & Why
Create dataset `netflix` and load six CSVs with **autodetect** for speed (we’ll enforce schemas later).

### Build Prompt (two cells)
**Cell A:** Create (idempotently) dataset `netflix` in US multi-region; if it exists, print a friendly message.  
**Cell B:** Load tables from `gs://$BUCKET_NAME/netflix/`:
`users, movies, watch_history, recommendation_logs, search_logs, reviews`
with `--skip_leading_rows=1 --autodetect --source_format=CSV`.
Finish with row-count queries for each table.


In [ ]:
DATASET="netflix"
# Attempt to create; ignore if exists
!bq --location=US mk -d --description "MGMT467 Netflix dataset" $DATASET || echo "Dataset may already exist."

BigQuery error in mk operation: Dataset 'mgmt467-4889:netflix' already exists.
Dataset may already exist.


In [ ]:
tables = {
  "users": "users.csv",
  "movies": "movies.csv",
  "watch_history": "watch_history.csv",
  "recommendation_logs": "recommendation_logs.csv",
  "search_logs": "search_logs.csv",
  "reviews": "reviews.csv",
}
import os
for tbl, fname in tables.items():
  src = f"gs://{os.environ['BUCKET_NAME']}/netflix/{fname}"
  print("Loading", tbl, "from", src)
  # Corrected bq load command
  !bq load --skip_leading_rows=1 --autodetect --source_format=CSV {DATASET}.{tbl} {src}

# Row counts
for tbl in tables.keys():
  # Construct the query string in Python before passing to bq query
  query = f"SELECT '{tbl}' AS table_name, COUNT(*) AS n FROM `{os.environ['GOOGLE_CLOUD_PROJECT']}.netflix.{tbl}`"
  !bq query --nouse_legacy_sql "{query}"

Loading users from gs://mgmt467-netflix-237da3ea/netflix/users.csv
Waiting on bqjob_r55d9f4ead82aed7_0000019a2128489b_1 ... (16s) Current status: DONE   
Loading movies from gs://mgmt467-netflix-237da3ea/netflix/movies.csv
Waiting on bqjob_r40949599260b6b82_0000019a2128a1df_1 ... (1s) Current status: DONE   
Loading watch_history from gs://mgmt467-netflix-237da3ea/netflix/watch_history.csv
Waiting on bqjob_red210fcf49bc149_0000019a2128c0d1_1 ... (16s) Current status: DONE   
Loading recommendation_logs from gs://mgmt467-netflix-237da3ea/netflix/recommendation_logs.csv
Waiting on bqjob_r194b8b74480d4dcc_0000019a212917aa_1 ... (1s) Current status: DONE   
Loading search_logs from gs://mgmt467-netflix-237da3ea/netflix/search_logs.csv
Waiting on bqjob_r6e6eec8134cb0daf_0000019a21293645_1 ... (2s) Current status: DONE   
Loading reviews from gs://mgmt467-netflix-237da3ea/netflix/reviews.csv
Waiting on bqjob_r5ffadf35c1247e0a_0000019a2129587a_1 ... (1s) Current status: DONE   
/bin/bash: lin

### Verification Prompt
Generate a single query that returns `table_name, row_count` for all six tables in `$mgmt467-4889.netflix`.


In [ ]:
%%bigquery
SELECT 'users' AS table_name, COUNT(*) AS row_count FROM `mgmt467-4889.netflix.users`
UNION ALL
SELECT 'movies' AS table_name, COUNT(*) AS row_count FROM `mgmt467-4889.netflix.movies`
UNION ALL
SELECT 'watch_history' AS table_name, COUNT(*) AS row_count FROM `mgmt467-4889.netflix.watch_history`
UNION ALL
SELECT 'recommendation_logs' AS table_name, COUNT(*) AS row_count FROM `mgmt467-4889.netflix.recommendation_logs`
UNION ALL
SELECT 'search_logs' AS table_name, COUNT(*) AS row_count FROM `mgmt467-4889.netflix.search_logs`
UNION ALL
SELECT 'reviews' AS table_name, COUNT(*) AS row_count FROM `mgmt467-4889.netflix.reviews`

Query is running:   0%|          |

Downloading:   0%|          |

,table_name,row_count
0,recommendation_logs,156000
1,reviews,46350
2,movies,3120
3,search_logs,79500
4,users,30900
5,watch_history,315000


In [ ]:
%%bigquery
SELECT column_name, data_type
FROM `mgmt467-4889.netflix.INFORMATION_SCHEMA.COLUMNS`
WHERE table_name = 'users'

Query is running:   0%|          |

Downloading:   0%|          |

,column_name,data_type
0,user_id,STRING
1,email,STRING
2,first_name,STRING
3,last_name,STRING
4,age,FLOAT64
5,gender,STRING
6,country,STRING
7,state_province,STRING
8,city,STRING
9,subscription_plan,STRING


In [ ]:
%load_ext google.cloud.bigquery

/usr/local/lib/python3.12/dist-packages/google/cloud/bigquery/__init__.py:237: FutureWarning: %load_ext google.cloud.bigquery is deprecated. Install bigquery-magics package and use `%load_ext bigquery_magics`, instead.
  warnings.warn(


In [ ]:
%load_ext bigquery_magics

The bigquery_magics extension is already loaded. To reload it, use:
  %reload_ext bigquery_magics


**Reflection:** When is `autodetect` acceptable? When should you enforce explicit schemas and why?

We can use `autodetect` in BigQuery when we're just doing some quick exploration or initial loading and aren't worried about perfectly clean data types yet. But, when we're building a solid data pipeline for production or analysis, we should totally enforce explicit schemas. That way, we make sure our data types are exactly what we expect, which prevents weird errors and keeps our data reliable for downstream tasks.

## 5) Data Quality (DQ) — Concepts we care about
- **Missingness** (MCAR/MAR/MNAR). Impute vs drop. Add `is_missing_*` indicators.
- **Duplicates** (exact vs near). Double-counted engagement corrupts labels & KPIs.
- **Outliers** (IQR). Winsorize/cap vs robust models. Always **flag** and explain.
- **Reproducibility**. Prefer `CREATE OR REPLACE` and deterministic keys.


### 5.1 Missingness (users) — What & Why
Measure % missing and check if missingness depends on another variable (MAR) → potential bias & instability.

### Build Prompt
Generate **two BigQuery SQL cells**:
1) Total rows and % missing in `region`, `plan_tier`, `age_band` from `users`.
2) `% plan_tier missing by region` ordered descending. Add comments on MAR.


In [ ]:
%%bigquery
WITH base AS (
  SELECT COUNT(*) n,
         COUNTIF(country IS NULL) miss_country, # no column names 'region'
         COUNTIF(subscription_plan IS NULL) miss_plan, # no column named 'plan_tier'
         COUNTIF(age IS NULL) miss_age
  FROM `mgmt467-4889.netflix.users`
)
SELECT n, miss_country, miss_plan, miss_age # Added SELECT statement
FROM base; # Removed ORDER BY DESC

Query is running:   0%|          |

Downloading:   0%|          |

,n,miss_country,miss_plan,miss_age
0,30900,0,0,3687


In [ ]:
%%bigquery
SELECT
    country,
    ROUND(100 * COUNTIF(country IS NULL) / COUNT(*), 2) AS pct_missing_country,
    ROUND(100 * COUNTIF(subscription_plan IS NULL) / COUNT(*), 2) AS pct_missing_subscription_plan,
    ROUND(100 * COUNTIF(age IS NULL) / COUNT(*), 2) AS pct_missing_age
FROM
    `mgmt467-4889.netflix.users`
GROUP BY
    country
ORDER BY
    country;

Query is running:   0%|          |

Downloading:   0%|          |

,country,pct_missing_country,pct_missing_subscription_plan,pct_missing_age
0,Canada,0.0,0.0,11.95
1,USA,0.0,0.0,11.92


### Verification Prompt
Generate a query that prints the three missingness percentages from (1), rounded to two decimals.


In [ ]:
%%bigquery
WITH base AS (
  SELECT COUNT(*) n,
         COUNTIF(country IS NULL) miss_country, # no column names 'region'
         COUNTIF(subscription_plan IS NULL) miss_plan, # no column named 'plan_tier'
         COUNTIF(age IS NULL) miss_age
  FROM `mgmt467-4889.netflix.users`
)
SELECT n,
       ROUND(100*miss_country/n,2) AS pct_missing_country,
       ROUND(100*miss_plan/n,2)   AS pct_missing_subscription_plan,
       ROUND(100*miss_age/n,2)    AS pct_missing_age
FROM base;

Query is running:   0%|          |

Downloading:   0%|          |

,n,pct_missing_country,pct_missing_subscription_plan,pct_missing_age
0,30900,0.0,0.0,11.93


**Reflection:** Which columns are most missing? Hypothesize MCAR/MAR/MNAR and why.

The age column is the most missing one for us, with about 11.93% of the data gone. Country and subscription_plan are complete, which is probably more important for regulations and actually making money.

Thinking about why age might be missing:

MCAR: The reason it's missing has nothing to do with age or anything else we know about the user, it's by chance.
MAR: This means the missing age is linked to something else we know. Maybe people in Canada were less likely to give their age? But our numbers by country looked pretty close, so maybe not strongly MAR based on country here.
MNAR: This is when the missing values are because of the age itself. Maybe teenagers or really old people didn't want to share their age. We can't really tell this from just looking at the data we have.

### 5.2 Duplicates (watch_history) — What & Why
Find exact duplicate interaction records and keep **one best** per group (deterministic policy).

### Build Prompt
Generate **two BigQuery SQL cells**:
1) Report duplicate groups on `(user_id, movie_id, event_ts, device_type)` with counts (top 20).
2) Create table `watch_history_dedup` that keeps one row per group (prefer higher `progress_ratio`, then `minutes_watched`). Add comments.


In [ ]:
%%bigquery
SELECT column_name, data_type
FROM `mgmt467-4889.netflix.INFORMATION_SCHEMA.COLUMNS`
WHERE table_name = 'watch_history'

Query is running:   0%|          |

Downloading:   0%|          |

,column_name,data_type
0,session_id,STRING
1,user_id,STRING
2,movie_id,STRING
3,watch_date,DATE
4,device_type,STRING
5,watch_duration_minutes,FLOAT64
6,progress_percentage,FLOAT64
7,action,STRING
8,quality,STRING
9,location_country,STRING


In [ ]:
# get all column names from all tables in mgmt467-4489.netflix
%%bigquery
SELECT column_name, data_type
FROM `mgmt467-4889.netflix.INFORMATION_SCHEMA.COLUMNS`
# order in abc order
ORDER BY column_name ASC

Query is running:   0%|          |

Downloading:   0%|          |

,column_name,data_type
0,action,STRING
1,active_next_month,INT64
2,added_to_platform,DATE
3,age,FLOAT64
4,age,FLOAT64
...,...,...
117,user_rating,INT64
118,was_clicked,BOOL
119,watch_date,DATE
120,watch_duration_minutes,FLOAT64


In [ ]:
%%bigquery
SELECT user_id, movie_id, watch_date, device_type, COUNT(*) AS dup_count
# event_ts isn't a column but watch_date is
FROM `mgmt467-4889.netflix.watch_history`
GROUP BY user_id, movie_id, watch_date, device_type
HAVING dup_count > 1
ORDER BY dup_count DESC
LIMIT 20;

Query is running:   0%|          |

Downloading:   0%|          |

,user_id,movie_id,watch_date,device_type,dup_count
0,user_03310,movie_0640,2024-09-08,Smart TV,12
1,user_00391,movie_0893,2024-08-26,Laptop,12
2,user_02950,movie_0928,2025-06-03,Desktop,9
3,user_09564,movie_0552,2025-01-11,Laptop,9
4,user_09815,movie_0827,2024-05-25,Laptop,9
5,user_00249,movie_0203,2024-08-31,Laptop,9
6,user_08681,movie_0332,2024-06-13,Laptop,9
7,user_01469,movie_0237,2025-01-17,Laptop,9
8,user_01383,movie_0015,2025-04-29,Desktop,9
9,user_04506,movie_0244,2025-05-27,Desktop,9


In [ ]:
%%bigquery
CREATE OR REPLACE TABLE `mgmt467-4889.netflix.watch_history_dedup` AS
SELECT * EXCEPT(rk) FROM (
  SELECT h.*,
         ROW_NUMBER() OVER (
           PARTITION BY user_id, movie_id, watch_date, device_type
           ORDER BY progress_percentage DESC, watch_duration_minutes DESC
         ) AS rk
  FROM `mgmt467-4889.netflix.watch_history` h
)
WHERE rk = 1;

Query is running:   0%|          |

""


In [ ]:
%%bigquery
SELECT *
FROM `mgmt467-4889.netflix.watch_history_dedup`
LIMIT 10

Query is running:   0%|          |

Downloading:   0%|          |

,session_id,user_id,movie_id,watch_date,device_type,watch_duration_minutes,progress_percentage,action,quality,location_country,is_download,user_rating
0,session_049455,user_09205,movie_0001,2025-10-18,Desktop,31.9,68.9,started,4K,Canada,False,<NA>
1,session_030873,user_08644,movie_0001,2025-11-05,Desktop,23.2,51.6,started,HD,USA,False,4
2,session_025600,user_02872,movie_0001,2025-03-13,Desktop,77.2,39.0,stopped,4K,USA,True,<NA>
3,session_044694,user_08786,movie_0001,2025-08-10,Desktop,84.4,38.1,stopped,HD,USA,False,<NA>
4,session_097143,user_02007,movie_0001,2024-01-26,Desktop,119.6,66.4,completed,SD,Canada,True,<NA>
5,session_037857,user_00786,movie_0001,2024-06-30,Desktop,101.9,NaN,stopped,HD,USA,False,<NA>
6,session_034354,user_08018,movie_0001,2024-09-30,Desktop,5.8,34.3,stopped,4K,USA,False,<NA>
7,session_064954,user_07749,movie_0001,2024-10-19,Desktop,NaN,1.4,paused,4K,Canada,False,<NA>
8,session_022397,user_05100,movie_0001,2024-04-24,Desktop,83.2,61.6,paused,SD,USA,False,<NA>
9,session_016536,user_08115,movie_0001,2025-03-26,Desktop,15.2,76.4,started,HD,USA,False,<NA>


### Verification Prompt
Generate a before/after count query comparing raw vs `watch_history_dedup`.


In [ ]:
%%bigquery
SELECT
  (SELECT COUNT(*) FROM `mgmt467-4889.netflix.watch_history`) AS original_row_count,
  (SELECT COUNT(*) FROM `mgmt467-4889.netflix.watch_history_dedup`) AS deduped_row_count

Query is running:   0%|          |

Downloading:   0%|          |

,original_row_count,deduped_row_count
0,315000,100000


**Reflection:** Why do duplicates arise (natural vs system-generated)? How do they corrupt labels and KPIs?

Duplicates can pop up for a couple of reasons. Sometimes it's just how the data naturally gets collected like a user accidentally clicking something twice, and other times it's due to system glitches or how data is processed. When we have duplicates, it can mess up our labels and KPIs. For example, if we're counting how many times a movie was watched, duplicates would inflate that number, giving us a false sense of engagement. This can lead to bad decisions based on inaccurate metrics.

### 5.3 Outliers (minutes_watched) — What & Why
Estimate extreme values via IQR; report % outliers; **winsorize** to P01/P99 for robustness while also **flagging** extremes.

### Build Prompt
Generate **two BigQuery SQL cells**:
1) Compute IQR bounds for `minutes_watched` on `watch_history_dedup` and report % outliers.
2) Create `watch_history_robust` with `minutes_watched_capped` capped at P01/P99; return quantile summaries before/after.


In [ ]:
%%bigquery
SELECT column_name, data_type
FROM `mgmt467-4889.netflix.INFORMATION_SCHEMA.COLUMNS`
WHERE table_name = 'watch_history_dedup'

Query is running:   0%|          |

Downloading:   0%|          |

,column_name,data_type
0,session_id,STRING
1,user_id,STRING
2,movie_id,STRING
3,watch_date,DATE
4,device_type,STRING
5,watch_duration_minutes,FLOAT64
6,progress_percentage,FLOAT64
7,action,STRING
8,quality,STRING
9,location_country,STRING


In [ ]:
%%bigquery
WITH dist AS (
  SELECT
    APPROX_QUANTILES(watch_duration_minutes, 4)[OFFSET(1)] AS q1,
    APPROX_QUANTILES(watch_duration_minutes, 4)[OFFSET(3)] AS q3
  FROM `mgmt467-4889.netflix.watch_history_dedup`
),
bounds AS (
  SELECT q1, q3, (q3-q1) AS iqr,
         q1 - 1.5*(q3-q1) AS lo,
         q3 + 1.5*(q3-q1) AS hi
  FROM dist
)
SELECT
  COUNTIF(h.watch_duration_minutes < b.lo OR h.watch_duration_minutes > b.hi) AS outliers,
  COUNT(*) AS total,
  ROUND(100*COUNTIF(h.watch_duration_minutes < b.lo OR h.watch_duration_minutes > b.hi)/COUNT(*),2) AS pct_outliers
FROM `mgmt467-4889.netflix.watch_history_dedup` h
CROSS JOIN bounds b;

Query is running:   0%|          |

Downloading:   0%|          |

,outliers,total,pct_outliers
0,3433,100000,3.43


In [ ]:
%%bigquery
CREATE OR REPLACE TABLE `mgmt467-4889.netflix.watch_history_robust` AS
WITH q AS (
  SELECT
    APPROX_QUANTILES(watch_duration_minutes, 100)[OFFSET(1)]  AS p01,
    APPROX_QUANTILES(watch_duration_minutes, 100)[OFFSET(98)] AS p99
  FROM `mgmt467-4889.netflix.watch_history_dedup`
)
SELECT
  h.*,
  GREATEST(q.p01, LEAST(q.p99, h.watch_duration_minutes)) AS watch_duration_minutes_capped
FROM `mgmt467-4889.netflix.watch_history_dedup` h, q;

-- Quantiles before vs after
WITH before AS (
  SELECT 'before' AS which, APPROX_QUANTILES(watch_duration_minutes, 5) AS q
  FROM `mgmt467-4889.netflix.watch_history_dedup`
),
after AS (
  SELECT 'after' AS which, APPROX_QUANTILES(watch_duration_minutes_capped, 5) AS q
  FROM `mgmt467-4889.netflix.watch_history_robust`
)
SELECT * FROM before UNION ALL SELECT * FROM after;

Query is running:   0%|          |

Downloading:   0%|          |

,which,q
0,after,"[4.4, 24.6, 41.5, 61.5, 92.0, 204.0]"
1,before,"[0.2, 25.0, 41.8, 61.1, 91.9, 799.3]"


### Verification Prompt
Generate a query that shows min/median/max before vs after capping.


In [ ]:
%%bigquery
SELECT
  'before' as type,
  min(watch_duration_minutes) as min_duration,
  approx_quantiles(watch_duration_minutes, 2)[OFFSET(1)] as median_duration,
  max(watch_duration_minutes) as max_duration
FROM `mgmt467-4889.netflix.watch_history_dedup`
UNION ALL
SELECT
  'after' as type,
  min(watch_duration_minutes_capped) as min_duration,
  approx_quantiles(watch_duration_minutes_capped, 2)[OFFSET(1)] as median_duration,
  max(watch_duration_minutes_capped) as max_duration
FROM `mgmt467-4889.netflix.watch_history_robust`

Query is running:   0%|          |

Downloading:   0%|          |

,type,min_duration,median_duration,max_duration
0,before,0.2,51.4,799.3
1,after,4.4,51.4,204.0


**Reflection:** When might capping be harmful? Name a model type less sensitive to outliers and why.

If the outliers represent genuine, important data points that carry out significant information. If these reflect real world phenomena, capping them could lead to a loss of valuable information. Tree based models are generally less sensitive to outliers compared to models like linear regression or k-nearest neighbors because tree based models make decisions by splitting data at threshholds, so regardless how far away from the threshhold a data point is there is no significant impact.

### 5.4 Business anomaly flags — What & Why
Human-readable flags help both product decisioning and ML features (e.g., binge behavior).

### Build Prompt
Generate **three BigQuery SQL cells** (adjust if columns differ):
1) In `watch_history_robust`, compute and summarize `flag_binge` for sessions > 8 hours.
2) In `users`, compute and summarize `flag_age_extreme` if age can be parsed from `age_band` (<10 or >100).
3) In `movies`, compute and summarize `flag_duration_anomaly` where `duration_min` < 15 or > 480 (if exists).
Each cell should output count and percentage and include 1–2 comments.


In [ ]:
%%bigquery
SELECT column_name, data_type
FROM `mgmt467-4889.netflix.INFORMATION_SCHEMA.COLUMNS`
WHERE table_name = 'watch_history_robust'

Query is running:   0%|          |

Downloading:   0%|          |

,column_name,data_type
0,session_id,STRING
1,user_id,STRING
2,movie_id,STRING
3,watch_date,DATE
4,device_type,STRING
5,watch_duration_minutes,FLOAT64
6,progress_percentage,FLOAT64
7,action,STRING
8,quality,STRING
9,location_country,STRING


In [ ]:
%%bigquery
SELECT
  COUNTIF(watch_duration_minutes > 8*60) AS sessions_over_8h,
  COUNT(*) AS total,
  ROUND(100*COUNTIF(watch_duration_minutes > 8*60)/COUNT(*),2) AS pct
FROM `mgmt467-4889.netflix.watch_history_robust`;

Query is running:   0%|          |

Downloading:   0%|          |

,sessions_over_8h,total,pct
0,639,100000,0.64


In [ ]:
%%bigquery
SELECT column_name, data_type
FROM `mgmt467-4889.netflix.INFORMATION_SCHEMA.COLUMNS`
WHERE table_name = 'users'

Query is running:   0%|          |

Downloading:   0%|          |

,column_name,data_type
0,user_id,STRING
1,email,STRING
2,first_name,STRING
3,last_name,STRING
4,age,FLOAT64
5,gender,STRING
6,country,STRING
7,state_province,STRING
8,city,STRING
9,subscription_plan,STRING


In [ ]:
%%bigquery
SELECT
  COUNTIF(CAST(REGEXP_EXTRACT(CAST(age AS STRING), r'\d+') AS INT64) < 10 OR
          CAST(REGEXP_EXTRACT(CAST(age AS STRING), r'\d+') AS INT64) > 100) AS extreme_age_rows,
  COUNT(*) AS total,
  ROUND(100*COUNTIF(CAST(REGEXP_EXTRACT(CAST(age AS STRING), r'\d+') AS INT64) < 10 OR
                    CAST(REGEXP_EXTRACT(CAST(age AS STRING), r'\d+') AS INT64) > 100)/COUNT(*),2) AS pct
FROM `mgmt467-4889.netflix.users`;

Query is running:   0%|          |

Downloading:   0%|          |

,extreme_age_rows,total,pct
0,537,30900,1.74


In [ ]:
%%bigquery
SELECT column_name, data_type
FROM `mgmt467-4889.netflix.INFORMATION_SCHEMA.COLUMNS`
WHERE table_name = 'movies'

Query is running:   0%|          |

Downloading:   0%|          |

,column_name,data_type
0,movie_id,STRING
1,title,STRING
2,content_type,STRING
3,genre_primary,STRING
4,genre_secondary,STRING
5,release_year,INT64
6,duration_minutes,FLOAT64
7,rating,STRING
8,language,STRING
9,country_of_origin,STRING


In [ ]:
%%bigquery
SELECT
  COUNTIF(duration_minutes < 15) AS titles_under_15m,
  COUNTIF(duration_minutes > 8*60) AS titles_over_8h,
  COUNT(*) AS total
FROM `mgmt467-4889.netflix.movies`;

Query is running:   0%|          |

Downloading:   0%|          |

,titles_under_15m,titles_over_8h,total
0,36,33,3120


### Verification Prompt
Generate a single compact summary query that returns two columns per flag: `flag_name, pct_of_rows`.


In [ ]:
%%bigquery
SELECT 'flag_binge' AS flag_name, ROUND(100*COUNTIF(watch_duration_minutes > 8*60)/COUNT(*),2) AS pct_of_rows
FROM `mgmt467-4889.netflix.watch_history_robust`
UNION ALL
SELECT 'flag_age_extreme' AS flag_name, ROUND(100*COUNTIF(CAST(REGEXP_EXTRACT(CAST(age AS STRING), r'\d+') AS INT64) < 10 OR CAST(REGEXP_EXTRACT(CAST(age AS STRING), r'\d+') AS INT64) > 100)/COUNT(*),2) AS pct_of_rows
FROM `mgmt467-4889.netflix.users`
UNION ALL
SELECT 'flag_duration_anomaly' AS flag_name, ROUND(100*COUNTIF(duration_minutes < 15 OR duration_minutes > 480)/COUNT(*),2) AS pct_of_rows
FROM `mgmt467-4889.netflix.movies`;

Query is running:   0%|          |

Downloading:   0%|          |

,flag_name,pct_of_rows
0,flag_binge,0.64
1,flag_age_extreme,1.74
2,flag_duration_anomaly,2.21


**Reflection:** Which anomaly flag is most common? Which would you keep as a feature and why?

The most common flag is flag duration anomaly. I think if anything, I would keep flag binge which identifies sessions where users watched for an unusually long time. This could be a useful feature for models that predict user engagement, churn (maybe binge watchers are less likely to churn soon?), or even content recommendations. Problem is from a health standpoint I'm not sure encourging bingers to continue binge watching is the greatest thing in the world but from a business perspective it makes sense.

## 6) Save & submit — What & Why
Reproducibility: save artifacts and document decisions so others can rerun and audit.

### Build Prompt
Generate a checklist (Markdown) students can paste at the end:
- Save this notebook to the team Drive.
- Export a `.sql` file with your DQ queries and save to repo.
- Push notebook + SQL to the **team GitHub** with a descriptive commit.
- Add a README with your `PROJECT_ID`, `REGION`, bucket, dataset, and today’s row counts.


## Grading rubric (quick)
- Profiling completeness (30)  
- Cleaning policy correctness & reproducibility (40)  
- Reflection/insight (20)  
- Hygiene (naming, verification, idempotence) (10)
